#Creating Data For Modeling


In [3]:
import sys
import numpy as np
import pandas as pd
from multiprocessing import Process
sys.path.insert(0, '../../')

In [4]:
from library.notebook_api.data_loader import CombinedDataLoader, ModelDataLoader

In [5]:
from library.source_data.feature_extractor import AudioFeatureExtractor
from library.source_data.parallel_processor import AudioParallelProcessor


## Balanced Extraction
This scenario runs extraction on a subset of the large version of the data set, filtering to rows matching a list of provided genres, and also trying to balance the dataset through obtaining an equal sized sample from each genre 

In [6]:
#instantate data_loader and the dataframes it makes available 
in_scope_labels = ['rock', 'electronic', 'hiphop', 'classical', 'jazz','country']
data_loader = CombinedDataLoader('large', in_scope_labels)
#df_balanced = data_loader.get_data_sampled_by_label(500)
df_balanced = data_loader.get_data_sampled_by_label(5000)

len(df_balanced)

tracks in meta 29701
tracks with files available in project_data_path:  29701
tracks with top level genres available 29701
tracks with genres and files (df_filtered) 29701


15947

to update using incremental batches, the code below checks the current context of the Model data version and determines the list of tracks that have yet to been process from the source

In [18]:
#get dataframe of current existing track ids from ModelDataLoader
current_data = ModelDataLoader('006')
current_data_df = current_data.df
current_track_list = current_data_df[['track_id','file_available']].copy()
#rename file_available column to differentiate it 
current_track_list.rename(columns={'file_available':'file_available_in_input'}, inplace=True)
#join this to medium source data to figure out tracks we don't have 
df_filtered_with_current_track_list = pd.merge(df_balanced,current_track_list,on='track_id',how='left')
#boolean indexer for the incremental files
incremental = (df_filtered_with_current_track_list['file_available_in_input'].isnull()) &(df_filtered_with_current_track_list['file_available'] ==1)

df_filtered_incremental = df_filtered_with_current_track_list[incremental].copy()

print(len(df_filtered_incremental), 'files left to process ')

2116 files left to process 


In [19]:
SAMPLE_RATE  = 22500
SECONDS = 25
#these batches include mfccs and log_melspectrogram, downsampled to 22500 , truncated to 25 seconds 
#batch_mfcc = AudioParallelProcessor(df_filtered_incremental.iloc[0:1200],version = '006', batch=2, threads = 4,sample_rate=SAMPLE_RATE,start_sample=0,end_sample=SAMPLE_RATE*SECONDS)
#batch_mfcc2 = AudioParallelProcessor(df_filtered_incremental.iloc[1201:],version = '006', batch=3, threads = 4,sample_rate=SAMPLE_RATE,start_sample=0,end_sample=SAMPLE_RATE*SECONDS)
#batch_mfcc = AudioParallelProcessor(df_filtered_incremental.iloc[0:2000],version = '006', batch=4, threads = 5,sample_rate=SAMPLE_RATE,start_sample=0,end_sample=SAMPLE_RATE*SECONDS)
#batch_mfcc2 = AudioParallelProcessor(df_filtered_incremental.iloc[2001:4000],version = '006', batch=5, threads = 5,sample_rate=SAMPLE_RATE,start_sample=0,end_sample=SAMPLE_RATE*SECONDS)

#batch_mfcc = AudioParallelProcessor(df_filtered_incremental.iloc[0:2000],version = '006', batch=6, threads = 5,sample_rate=SAMPLE_RATE,start_sample=0,end_sample=SAMPLE_RATE*SECONDS)
#batch_mfcc2 = AudioParallelProcessor(df_filtered_incremental.iloc[2001:4000],version = '006', batch=7, threads = 5,sample_rate=SAMPLE_RATE,start_sample=0,end_sample=SAMPLE_RATE*SECONDS)

#batch_mfcc = AudioParallelProcessor(df_filtered_incremental.iloc[0:2000],version = '006', batch=8, threads = 5,sample_rate=SAMPLE_RATE,start_sample=0,end_sample=SAMPLE_RATE*SECONDS)
#batch_mfcc2 = AudioParallelProcessor(df_filtered_incremental.iloc[2001:4000],version = '006', batch=9, threads = 5,sample_rate=SAMPLE_RATE,start_sample=0,end_sample=SAMPLE_RATE*SECONDS)

#batch_mfcc = AudioParallelProcessor(df_filtered_incremental.iloc[0:2000],version = '006', batch=10, threads = 5,sample_rate=SAMPLE_RATE,start_sample=0,end_sample=SAMPLE_RATE*SECONDS)
#batch_mfcc2 = AudioParallelProcessor(df_filtered_incremental.iloc[2001:4000],version = '006', batch=11, threads = 5,sample_rate=SAMPLE_RATE,start_sample=0,end_sample=SAMPLE_RATE*SECONDS)

batch_mfcc = AudioParallelProcessor(df_filtered_incremental,version = '006', batch=12, threads = 5,sample_rate=SAMPLE_RATE,start_sample=0,end_sample=SAMPLE_RATE*SECONDS)


In [20]:
batch_mfcc.execute()

populate thread process for thread 1  0 423 423
populate thread process for thread 2  424 846 422
populate thread process for thread 3  847 1269 422
populate thread process for thread 4  1270 1692 422
populate thread process for thread 5  1693 2116 423
start process thread 1
start process thread 2


python(60586) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


start process thread 3


python(60587) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


start process thread 4


...

/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/mu

In [17]:
batch_mfcc2.execute()

populate thread process for thread 1  0 399 399
populate thread process for thread 2  400 798 398
populate thread process for thread 3  799 1197 398
populate thread process for thread 4  1198 1596 398
populate thread process for thread 5  1597 1999 402
start process thread 1
start process thread 2
start process thread 3
start process thread 4
start process thread 5


[src/libmpg123/layer3.c:INT123_do_layer3():1801] error: dequantization failed!
[src/libmpg123/layer3.c:INT123_do_layer3():1841] error: dequantization failed!
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/00_featured_notebooks/../../library/source_data/feature_extractor.py:66: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(PROJECT_ABSOLUTE_PATH+file_name)
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


failure in librosa.load


[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
Note: Illegal Audio-MPEG-Header 0x00000000 at offset 33361.
Note: Trying to resync...
Note: Skipped 1024 bytes in input.
[src/libmpg123/parse.c:wetwork():1365] error: Giving up resync after 1024 bytes - your stream is not nice... (maybe increasing resync limit could help).
Note: Illegal Audio-MPEG-Header 0x00000000 at offset 187493.
Note: Trying to resync...
Note: Skipped 1024 bytes in input.
[src/libmpg123/parse.c:wetwork():1365] error: Giving up resync after 1024 bytes - your stream is not nice... (maybe increasing resync limit could help).


failure in librosa.load
failure in librosa.load
failure in librosa.load
failure in librosa.load


...

/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=558
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=720
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=360
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=180
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_cap

Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/mu

## Feature Extraction fma_large

### Load Metadata
pass the optional fma_audio size argument to Combined data loader to reference the fma_large file directory as part of combinding fma data with gtzan data

In [5]:
#instantate data_loader and the dataframes it makes available 
data_loader = CombinedDataLoader('large')
df = data_loader.df
df_files_available = data_loader.df_files_available
df_genres_available = data_loader.df_genres_available
df_filtered = data_loader.df_filtered

#get dataframe of current existing track ids from ModelDataLoader
current_data = ModelDataLoader('005')
current_data_df = current_data.df
current_track_list = current_data_df[['track_id','file_available']].copy()
#rename file_available column to differentiate it 
current_track_list.rename(columns={'file_available':'file_available_in_input'}, inplace=True)
#join this to medium source data to figure out tracks we don't have 
df_filtered_with_current_track_list = pd.merge(df_filtered,current_track_list,on='track_id',how='left')
#boolean indexer for the incremental files
incremental = (df_filtered_with_current_track_list['file_available_in_input'].isnull()) &(df_filtered_with_current_track_list['file_available'] ==1)

df_filtered_incremental = df_filtered_with_current_track_list[incremental].copy()

print(len(df_filtered_incremental), 'files left to process ')

tracks in meta 107574
tracks with files available in project_data_path:  107574
tracks with top level genres available 50598
tracks with genres and files (df_filtered) 50598
25720 files left to process 


In [ ]:
in_scope_labels = ['rock', 'electronic', 'hiphop', 'classical', 'jazz','country']

df_incremental_under_represented = df_filtered_incremental[df_filtered_incremental['label'].apply(lambda label: True if label in in_scope_labels else False)]
df_incremental_under_represented['label'].value_counts()

label
hiphop       2173
classical     444
jazz          255
country       202
Name: count, dtype: int64

### Extract and save numerical features in parquet and mfcc as npy nd array

In [10]:
batch_mfcc = AudioParallelProcessor(df_incremental_under_represented,version = '005', batch=12, threads = 4)

#batch_mfcc = AudioParallelProcessor(df_filtered_incremental.iloc[0:2000],version = '005', batch=11, threads = 4)
#batch_mfcc2 = AudioParallelProcessor(df_filtered_incremental.iloc[2001:4000],version = '005', batch=12, threads = 4)
#batch_mfcc3 = AudioParallelProcessor(df_filtered_incremental.iloc[4001:6000],version = '005', batch=13, threads = 4)

In [11]:
batch_mfcc.execute()

populate thread process for thread 1  0 768 768
populate thread process for thread 2  769 1536 767
populate thread process for thread 3  1537 2304 767
populate thread process for thread 4  2305 3074 769
start process thread 1
start process thread 2
start process thread 3
start process thread 4


[src/libmpg123/layer3.c:INT123_do_layer3():1801] error: dequantization failed!
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:58: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(PROJECT_ABSOLUTE_PATH+file_name, sr=None)
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


failure in librosa.load


[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
Note: Illegal Audio-MPEG-Header 0x00000000 at offset 33361.
Note: Trying to resync...
Note: Skipped 1024 bytes in input.
[src/libmpg123/parse.c:wetwork():1365] error: Giving up resync after 1024 bytes - your stream is not nice... (maybe increasing resync limit could help).
Note: Illegal Audio-MPEG-Header 0x00000000 at offset 187493.
Note: Trying to resync...
Note: Skipped 1024 bytes in input.
[src/libmpg123/parse.c:wetwork():1365] error: Giving up resync after 1024 bytes - your stream is not nice... (maybe increasing resync limit could help).
Note: Illegal Audio-MPEG-Header

failure in librosa.load
failure in librosa.load
failure in librosa.load
failure in librosa.load


...

/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:118: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)[0]
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:118: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)[0]
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:118: FutureWarning: librosa.beat.tempo
	This 

Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_save['fma_genres'] = to_save['fma_genres'].astype('string')
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_save['fma_genres_all'] = to_sa

In [20]:
batch_mfcc2.execute()

populate thread process for thread 1  0 499 499
populate thread process for thread 2  500 998 498
populate thread process for thread 3  999 1497 498
populate thread process for thread 4  1498 1999 501
start process thread 1


start process thread 2


python(71219) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


start process thread 3


python(71224) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


...

/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [21]:
batch_mfcc3.execute()

populate thread process for thread 1  0 499 499
populate thread process for thread 2  500 998 498
populate thread process for thread 3  999 1497 498
populate thread process for thread 4  1498 1999 501
start process thread 1
start process thread 2


python(72500) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


start process thread 3


python(72501) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(72502) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


start process thread 4


...

/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=944
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=967
  warnings.warn(


Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [22]:
m = ModelDataLoader('005')

In [23]:
m.df.head()
len(m.df)

24878